In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the path to the main dataset directory
main_dataset_path = 'Vowel_Dataset'

# Define the subdirectories (classes) inside the main dataset directory
classes = ['A', 'Aa', 'Ai', 'E', 'Ee', 'U']

# Define the paths for the training and testing datasets
train_dataset_path = 'train_set'
test_dataset_path = 'test_set'

# Create training and testing directories if they don't exist
os.makedirs(train_dataset_path, exist_ok=True)
os.makedirs(test_dataset_path, exist_ok=True)

# Iterate through each class and split the data into training and testing
for class_name in classes:
    class_path = os.path.join(main_dataset_path, class_name)
    images = os.listdir(class_path)
    
    # Split the images into training and testing sets
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    
    # Move images to the respective directories
    for img in train_images:
        src_path = os.path.join(class_path, img)
        dest_path = os.path.join(train_dataset_path, class_name, img)
        os.makedirs(os.path.join(train_dataset_path, class_name), exist_ok=True)
        shutil.copy(src_path, dest_path)
    
    for img in test_images:
        src_path = os.path.join(class_path, img)
        dest_path = os.path.join(test_dataset_path, class_name, img)
        os.makedirs(os.path.join(test_dataset_path, class_name), exist_ok=True)
        shutil.copy(src_path, dest_path)

print("Dataset split into training and testing sets.")


Dataset split into training and testing sets.


In [8]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths for the training and testing datasets
train_dataset_path = 'train_set'
test_dataset_path = 'test_set'

# Define image size and batch size
img_size = (300, 232)
batch_size = 32

# Create an ImageDataGenerator for data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2,
)

# Create an ImageDataGenerator for testing (no data augmentation)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
)

# Load and preprocess testing data
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
)

# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 232, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(train_generator.class_indices), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model and get the history object
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
)

# Print the training accuracy after each epoch
for epoch, acc in enumerate(history.history['accuracy']):
    print(f'Epoch {epoch + 1}/10 - Training Accuracy: {acc:.4f}')

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy}')

# Save the model
model.save('cnn_model.h5')


Found 960 images belonging to 6 classes.
Found 240 images belonging to 6 classes.
Epoch 1/10
30/30 [==============================] - 69s 2s/step - loss: 1.9389 - accuracy: 0.3219 - val_loss: 0.8068 - val_accuracy: 0.6417
Epoch 2/10
30/30 [==============================] - 69s 2s/step - loss: 1.0427 - accuracy: 0.5990 - val_loss: 0.4951 - val_accuracy: 0.7958
Epoch 3/10
30/30 [==============================] - 74s 2s/step - loss: 0.7381 - accuracy: 0.7344 - val_loss: 0.3871 - val_accuracy: 0.8208
Epoch 4/10
30/30 [==============================] - 74s 2s/step - loss: 0.6500 - accuracy: 0.7667 - val_loss: 0.3807 - val_accuracy: 0.8500
Epoch 5/10
30/30 [==============================] - 78s 3s/step - loss: 0.5986 - accuracy: 0.7917 - val_loss: 0.3157 - val_accuracy: 0.8958
Epoch 6/10
30/30 [==============================] - 86s 3s/step - loss: 0.5590 - accuracy: 0.7948 - val_loss: 0.2570 - val_accuracy: 0.9167
Epoch 7/10
30/30 [==============================] - 89s 3s/step - loss: 0.4696

In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Import ImageDataGenerator
import numpy as np

# Define image size and batch size
img_size = (300, 232)
batch_size = 32

# Load the trained model
model = load_model('cnn_model.h5')

# Define the image file path
image_path = 'test2.jpg'  # Replace 'test2.jpg' with the actual path to your image

# Load and preprocess the image
img = image.load_img(image_path, target_size=img_size)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the image

# Make predictions
predictions = model.predict(img_array)

# Load the training data generator to access class indices
train_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Define with appropriate preprocessing
train_generator = train_datagen.flow_from_directory(
    'train_set',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
)

# Get the class label from the generator's class indices
class_label = list(train_generator.class_indices.keys())[np.argmax(predictions)]

# Get the predicted probability
predicted_probability = predictions[0][np.argmax(predictions)]

# Print the predicted class label and probability
print(f'Predicted Class: {class_label} - Probability: {predicted_probability:.4f}')


Found 960 images belonging to 6 classes.
Predicted Class: Ee - Probability: 0.9999


In [3]:
import matplotlib.pyplot as plt

# Plotting training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plotting training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


NameError: name 'history' is not defined

In [2]:
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
epochs = range(1, len(training_accuracy) + 1)


NameError: name 'history' is not defined

ValueError: in user code:

    c:\users\sravyareddypati\desktop\django-ecommerce\test\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    c:\users\sravyareddypati\desktop\django-ecommerce\test\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\sravyareddypati\desktop\django-ecommerce\test\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\sravyareddypati\desktop\django-ecommerce\test\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\sravyareddypati\desktop\django-ecommerce\test\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, 